# Plot mean time-series across participants 
# - EA ratings, ecg, eeg and brain-hf features -
# for one head movement condition

### 2024 by Antonin Fourcade

In [16]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.io import loadmat, savemat

# Parameters
mov_cond = ['mov', 'nomov'] # head movement condition
c_style = 'SBA' # sequence o the rollercosaters: Space-Break-Andes
nb_samples = 270 # 1Hz ratings for SBA = 270s

#order of mov and nomov conditions in data structures
if mov_cond == 'nomov': 
    mov_idx = 0
elif mov_cond == 'mov':
    mov_idx = 1

data_path = 'E:/NeVRo/Data/'
data_path_ecg = data_path + 'ECG/'
data_path_eegpow = data_path + 'Frequency_Bands_Power/ROI_EEGpow/'
data_path_brainhf = data_path + 'Frequency_Bands_Power/ROI_BrainHF/'
aro_rat_z_path = data_path + 'ratings/continuous/z_scored/' + mov_cond + '/' + c_style + '/' 

rc_events = {
            'rapid_curve_spiral':47, #space event
            'looping':60, #space event
            'spiral':73, #space event
            'fire':20+178, #andes event
            'steep_fall':24+178, #andes event
            'jump_1':31+178, #andes event
            'landing_1':33+178, #andes event
            'fire_looping':51+178, #andes event
            'jump_2':67+178, #andes event
            'landing_2':72+178 #andes event
              }

# All mov are included in nomov, but nomov have more SJs
sjs = {
    "nomov": ['02', '04', '05', '06', '07', '08', '09', '11', '13', '14', '15', '17', '18', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '34', '36', '37', '39', '44'],
    "mov": ['02', '04', '05', '06', '08', '09', '11', '13', '14', '17', '18', '20', '21', '22', '24', '25', '27', '28', '29', '31', '34', '36', '37', '39', '44']   
      }

samples = np.arange(0,nb_samples)

all_toplot = {
    "aro_rat_z":np.empty((len(sjs[mov_cond]), nb_samples)),
    "ibi":np.empty((len(sjs[mov_cond]), nb_samples)),
    "hrv_lf":np.empty((len(sjs[mov_cond]), nb_samples)),
    "hrv_hf":np.empty((len(sjs[mov_cond]), nb_samples)),
    "alpha":np.empty((len(sjs[mov_cond]), nb_samples)),
    "alpha2hf":np.empty((len(sjs[mov_cond]), nb_samples)),
    "hf2alpha":np.empty((len(sjs[mov_cond]), nb_samples))
}

In [17]:
# Load data and select mov_cond
all_ecg_mat = loadmat(data_path_ecg + 'ECG.mat', simplify_cells=True)
all_ecg = all_ecg_mat[mov_cond]

all_eegpow_mat = loadmat(data_path_eegpow + 'EEG_pow.mat', simplify_cells=True)
all_eegpow = all_eegpow_mat[mov_cond]

all_brainhf_mat = loadmat(data_path_brainhf + 'C_BrainHF.mat', simplify_cells=True)
all_brainhf = all_brainhf_mat['C_BrainHF'][mov_idx]

files_aro_z = glob.glob(aro_rat_z_path + '*.txt')

In [18]:
for i, subj in enumerate(sjs[mov_cond]):
    all_toplot['aro_rat_z'][i] = pd.read_csv(files_aro_z[i], names=["idx", "ar", "latency"])["ar"]
    all_toplot['ibi'][i] = all_ecg[i]['rs_ibi']
    all_toplot['hrv_lf'][i] = all_ecg[i]['rs_hrv_lf']
    all_toplot['hrv_hf'][i] = all_ecg[i]['rs_hrv_hf']
    all_toplot['alpha'][i] = all_eegpow[i]['alpha_meanROI']
    all_toplot['alpha2hf'][i] = all_brainhf[i].Alpha2HF_meanROI
    all_toplot['hf2alpha'][i] = all_brainhf[i].HF2Alpha_meanROI

### Plot means

In [ ]:
f, ((ax1, ax4), (ax2, ax5), (ax3, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=[45,50])

# Parameters for plotting
plt.rcParams['figure.constrained_layout.use'] = False
parameters = {'axes.labelsize': 46, 'axes.titlesize': 48, 'font.size': 46}
plt.rcParams.update(parameters)
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['xtick.major.width'] = 3
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['ytick.major.size'] = 10
lw_plotting = 4
lw_break = 2
esiz = 32

# plot IBI
ibi_mean = all_toplot['ibi'].mean(0)
ibi_std = all_toplot['ibi'].std(0)
color = '#F97306'
ax1.plot(samples, ibi_mean, color=color, linewidth=lw_plotting)
ax1.fill_between(samples, ibi_mean - ibi_std, ibi_mean + ibi_std, color=color, alpha=.25)
ax1.axvline(x=148, color='grey', linestyle='dotted', linewidth=lw_break)
ax1.axvline(x=178, color='grey', linestyle='dotted', linewidth=lw_break)
ax1.set(xlabel='Time (s)', ylabel='mean IBI (s)')
ax1.set_xlim([0,270])

# plot LF-HRV
hrv_lf_mean = all_toplot['hrv_lf'].mean(0)
hrv_lf_std = all_toplot['hrv_lf'].std(0)
color = '#F97306'
ax2.plot(samples, hrv_lf_mean, color=color, linewidth=lw_plotting)
ax2.fill_between(samples, hrv_lf_mean - hrv_lf_std, hrv_lf_mean + hrv_lf_std, color=color, alpha=.25)
ax2.axvline(x=148, color='grey', linestyle='dotted', linewidth=lw_break)
ax2.axvline(x=178, color='grey', linestyle='dotted', linewidth=lw_break)
ax2.set(xlabel='Time (s)', ylabel='mean LF-HRV (a.u.)')
ax2.set_xlim([0,270])

# plot HF-HRV
hrv_hf_mean = all_toplot['hrv_hf'].mean(0)
hrv_hf_std = all_toplot['hrv_hf'].std(0)
color = '#F97306'
ax3.plot(samples, hrv_hf_mean, color=color, linewidth=lw_plotting)
ax3.fill_between(samples, hrv_hf_mean - hrv_hf_std, hrv_hf_mean + hrv_hf_std, color=color, alpha=.25)
ax3.axvline(x=148, color='grey', linestyle='dotted', linewidth=lw_break)
ax3.axvline(x=178, color='grey', linestyle='dotted', linewidth=lw_break)
ax3.set(xlabel='Time (s)', ylabel='mean HF-HRV (a.u.)')
ax3.set_xlim([0,270])

# plot EEG alpha power (log)
log_alpha_mean = np.log10(all_toplot['alpha']).mean(0)
log_alpha_std = np.log10(all_toplot['alpha']).std(0)
color = '#069AF3'
ax4.plot(samples, log_alpha_mean, color='#069AF3', linewidth=lw_plotting)
ax4.fill_between(samples, log_alpha_mean - log_alpha_std, log_alpha_mean + log_alpha_std, color=color, alpha=.25)
ax4.axvline(x=148, color='grey', linestyle='dotted', linewidth=lw_break)
ax4.axvline(x=178, color='grey', linestyle='dotted', linewidth=lw_break)
ax4.set(xlabel='Time (s)', ylabel='mean α power (log-transform)')
ax4.set_xlim([0,270])

# plot brain-to-heart coupling coefficients
alpha2hf_mean = all_toplot['alpha2hf'].mean(0)
alpha2hf_std = all_toplot['alpha2hf'].std(0)
color = '#00a9a9ff'
ax5.plot(samples, alpha2hf_mean, color=color, linewidth=lw_plotting)
ax5.fill_between(samples, alpha2hf_mean - alpha2hf_std, alpha2hf_mean + alpha2hf_std, color=color, alpha=.25)
ax5.axvline(x=148, color='grey', linestyle='dotted', linewidth=lw_break)
ax5.axvline(x=178, color='grey', linestyle='dotted', linewidth=lw_break)
ax5.set(xlabel='Time (s)', ylabel='mean α → HF-HRV (a.u.)')
ax5.set_xlim([0,270])

# plot heart-to-brain coupling coefficients
hf2alpha_mean = all_toplot['hf2alpha'].mean(0)
hf2alpha_std = all_toplot['hf2alpha'].std(0)
color = '#00a9a9ff'
ax6.plot(samples, hf2alpha_mean, color=color, linewidth=lw_plotting)
ax6.fill_between(samples, hf2alpha_mean - hf2alpha_std, hf2alpha_mean + hf2alpha_std, color=color, alpha=.25)
ax6.axvline(x=148, color='grey', linestyle='dotted', linewidth=lw_break)
ax6.axvline(x=178, color='grey', linestyle='dotted', linewidth=lw_break)
ax6.set(xlabel='Time (s)', ylabel='mean HF-HRV → α (a.u.)')
ax6.set_xlim([0,270])

# plot emotional arousal ratings (z-scores)
aro_mean = all_toplot['aro_rat_z'].mean(0)
aro_std = all_toplot['aro_rat_z'].std(0)
color = 'purple'
ax7.plot(samples, aro_mean, color=color, linewidth=lw_plotting)
ax7.fill_between(samples, aro_mean - aro_std, aro_mean + aro_std, color=color, alpha=.25)
ax7.axvline(x=148, color='grey', linestyle='dotted', linewidth=lw_break)
ax7.axvline(x=178, color='grey', linestyle='dotted', linewidth=lw_break)
ax7.set(xlabel='Time (s)', ylabel='mean Arousal ratings (z-scores)')
ax7.set_xlim([0,270])

# add rollercoaster events in plot
ylims = 2.1 #ylim for ratings
trim = 2 #2.5s trimming at beginning and end of rc

shift_counter = 0
for idxe, event in enumerate(rc_events):
    shift_counter += 1
    if shift_counter > 4:  # reset
        shift_counter = 1
    shift = 1 if shift_counter > 2 else 0  # switch between 1 and zero

    t_event = rc_events[event] - trim  # timepoint of event
    up_down = -1 if idxe % 2 == 0 else 1

    y_max_value = aro_mean[t_event]
    
    if up_down == -1:
        ax7.axvline(x=t_event, ymin=0.25, ymax=0.5)
    else:
        ax7.axvline(x=t_event, ymin=0.5, ymax=0.75)

    #ax7.axvline(x=t_event, ymin=(ylims - shift) * up_down, ymax=y_max_value/ylims)
               #linestyles="dotted",
               #alpha=1)
    ax7.text(x=t_event+1, y=(ylims - shift) * up_down, s=event.replace("_", " "), size=esiz)
#f.tight_layout()
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.3, 
                    hspace=0.4)
#plt.savefig('mean_timeseries' + mov_cond + '.png', dpi=600, facecolor='white', transparent=False)
plt.savefig('mean_timeseries' + mov_cond + '.svg')

#plt.rcdefaults() 